In [2]:
import warnings
import tensorflow as tf
import numpy as np
import pandas as pd
import json
#from modules.CARLA import carla

seed = 44

tf.random.set_seed(seed)
np.random.seed(seed)

In [3]:
# from carla.data.catalog import CsvCatalog
# import json

# with open('../data/adult_constraints.json', 'r') as f:
#     constraints = json.load(f)

# continuous = constraints['continuous']
# categorical = constraints['categorical']
# immutable = constraints['immutable']
# columns_order = constraints['features_order']

# dataset = CsvCatalog(file_path="../data/adult_prep.csv",
#                     continuous=continuous,
#                     categorical=categorical,
#                     immutables=immutable,
#                     target='income')
# dataset.df.to_csv("../data/adult_prep_test.csv",index=False)



# X_train = dataset.df_train
# Y_train = dataset.df_train
# X_test = dataset.df_test
# Y_test = dataset.df_test
# X_train

In [8]:
import json
import sklearn

with open('../data/adult_constraints.json', 'r') as f:
    constr = json.load(f)

dataset = pd.read_csv("../data/adult_cleaned.csv")

X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(dataset[constr['features_order_after_split']], dataset['income'], test_size=0.2, random_state=seed)
X_train.columns

Index(['capital.gain', 'education.num', 'capital.loss', 'hours.per.week',
       'age', 'Federal-gov', 'Local-gov', 'Never-worked', 'Private',
       'Self-emp-inc', 'Self-emp-not-inc', 'State-gov', 'Unknown_workclass',
       'Without-pay', 'Divorced', 'Married-AF-spouse', 'Married-civ-spouse',
       'Married-spouse-absent', 'Never-married', 'Separated', 'Widowed',
       'Adm-clerical', 'Armed-Forces', 'Craft-repair', 'Exec-managerial',
       'Farming-fishing', 'Handlers-cleaners', 'Machine-op-inspct',
       'Other-service', 'Priv-house-serv', 'Prof-specialty', 'Protective-serv',
       'Sales', 'Tech-support', 'Transport-moving', 'Unknown_occupation',
       'Amer-Indian-Eskimo', 'Asian-Pac-Islander', 'Black', 'Other', 'White',
       'Female', 'Male', 'Cambodia', 'Canada', 'China', 'Columbia', 'Cuba',
       'Dominican-Republic', 'Ecuador', 'El-Salvador', 'England', 'France',
       'Germany', 'Greece', 'Guatemala', 'Haiti', 'Holand-Netherlands',
       'Honduras', 'Hong', 'Hung

In [9]:
Y_train

20789     >50K
7665     <=50K
7003     <=50K
3368     <=50K
15072    <=50K
         ...  
16955    <=50K
25773     >50K
27377    <=50K
3491     <=50K
14100    <=50K
Name: income, Length: 26048, dtype: object

In [10]:

X_train = X_train.to_numpy()
Y_train = Y_train
Y_train = pd.get_dummies(Y_train).to_numpy()
Y_train

array([[0, 1],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]], dtype=uint8)

In [11]:
X_test = X_test.to_numpy()
Y_test = Y_test
Y_test = pd.get_dummies(Y_test).to_numpy()
Y_test

array([[1, 0],
       [1, 0],
       [1, 0],
       ...,
       [1, 0],
       [1, 0],
       [0, 1]], dtype=uint8)

In [12]:
X_train.shape

(26048, 85)

In [13]:
Y_train.shape

(26048, 2)

In [14]:
# sklearn minmaxscaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [15]:
constr['features_count_split_without_target']

85

In [16]:
from tensorflow import keras


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input((constr['features_count_split_without_target'],)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(2))
model.add(tf.keras.layers.Softmax())


model.compile(
    optimizer='rmsprop',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [17]:
model.fit(
    X_train, 
    Y_train,
    epochs=50,
    batch_size=256,
    validation_data=(X_test, Y_test),
    shuffle=True,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10)
    ]
)

Epoch 1/50
102/102 [==============================] - 1s 5ms/step - loss: 0.4451 - accuracy: 0.7906 - val_loss: 0.3852 - val_accuracy: 0.8267
Epoch 2/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3876 - accuracy: 0.8209 - val_loss: 0.3690 - val_accuracy: 0.8320
Epoch 3/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3715 - accuracy: 0.8312 - val_loss: 0.3579 - val_accuracy: 0.8342
Epoch 4/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3609 - accuracy: 0.8352 - val_loss: 0.3529 - val_accuracy: 0.8379
Epoch 5/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3535 - accuracy: 0.8387 - val_loss: 0.3470 - val_accuracy: 0.8399
Epoch 6/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3488 - accuracy: 0.8389 - val_loss: 0.3451 - val_accuracy: 0.8409
Epoch 7/50
102/102 [==============================] - 0s 3ms/step - loss: 0.3403 - accuracy: 0.8420 - val_loss: 0.3415 - val_accuracy: 0.8435
Epoch 

In [18]:
model.save('../models/adult_NN.h5', overwrite=True, save_format='h5')

In [19]:
model.save('../models/adult_NN', overwrite=True)
model_test = tf.keras.models.load_model('../models/adult_NN.h5')
model_test.predict(X_test[0:2], verbose=1)

1/1 [==============================] - 0s 91ms/step


array([[0.9115964 , 0.08840352],
       [0.9989188 , 0.0010813 ]], dtype=float32)

In [20]:
model.load_weights('../models/adult_NN.h5')

In [21]:
X_test[0].shape

(85,)

In [22]:
model.predict(X_test[0:2], verbose=1)


1/1 [==============================] - 0s 62ms/step


array([[0.9115964 , 0.08840352],
       [0.9989188 , 0.0010813 ]], dtype=float32)

RandomForest

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from joblib import dump, load

In [26]:
classifier = RandomForestClassifier(
    random_state=seed, 
    criterion='gini', 
    n_estimators=100, 
    max_depth=15, 
    min_samples_split=3, 
    min_samples_leaf=2, 
    min_weight_fraction_leaf=0.0, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    )
classifier.fit(X_train, Y_train)

RandomForestClassifier(max_depth=15, min_samples_leaf=2, min_samples_split=3,
                       random_state=44)

In [27]:
classifier.score(X_test, Y_test)

0.8593582066635959

In [28]:
dump(classifier, '../models/adult_RF.joblib')

['../models/adult_RF.joblib']

In [29]:
clf = load('../models/adult_RF.joblib')

In [30]:
print(Y_test[0:5])
clf.predict(X_test[0:5])

[[1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]]


array([[1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1]], dtype=uint8)